In [1]:
import tensorflow as tf
import pydot
from pathlib import Path
import os
import json
import requests
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

# Using Keras to load the dataset

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
fashion_mnist

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

In [ ]:
X_train, y_train =  X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid =  X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train.shape

In [ ]:
X_train.dtype

In [ ]:
# we'll scale the pixel intensities down to the 0-1 range
X_train, X_valid, X_test = X_train / 225., X_valid / 225., X_test / 225.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train[0]]

# Creating the model using the sequential API

In [ ]:
tf.random.set_seed(73) # tf.keras.utils.set_random_seed() sets the random seeds TensorFlow, Python and Numpy

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Input(shape=[28, 28]))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(300, activation="relu"))
# model.add(tf.keras.layers.Dense(100, activation="relu"))
# model.add(tf.keras.layers.Dense(10, activation="softmax"))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
# tf.keras.utils.plot_model(model)

Keras ensures the name a layers is globally unique, even across models. This makes it possible to merge models easily without getting name conflicts.

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
model.get_layer('dense') is hidden1

In [ ]:
weights, biases = hidden1.get_weights()
weights

In [ ]:
weights.shape

In [ ]:
biases

In [ ]:
biases.shape

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

## Model Checkpoint

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints", save_weights_only=True, save_best_only=True)

## Early Stopping

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version

In [ ]:
model.save(model_path, save_format="tf")

In [ ]:
!saved_model_cli show --dir my_mnist_model/0001

In [ ]:
!saved_model_cli show --dir my_mnist_model/0001 --tag_set serve

__saved_model_init_op --> an initialization function

serving_default --> defult serving function

In [ ]:
!saved_model_cli show --dir my_mnist_model/0001 --tag_set serve --signature_def serving_default

# TensorFlow Serving

Run install-tf-server.sh 

In [ ]:
%pip install -q -U tensorflow-serving-api

In [ ]:
os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [ ]:
%%bash --bg
tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=my_mnist_model --model_base_path="${MODEL_DIR}" > my_server.log 2>&1

It is now waiting for gRPC and REST requests, respectively, on ports 8500 and 8501

# Querying TF Serving through the REST API

In [ ]:
X_new = X_test[:6] # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [ ]:
server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

In [ ]:
y_test[:6]